In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Information.csv', encoding = 'latin-1')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

In [4]:
df = df.drop(['_unit_id','_golden','_last_judgment_at','_unit_state','_trusted_judgments','profile_yn','profile_yn:confidence','created','fav_number','gender_gold','link_color','name','profile_yn_gold','profileimage','retweet_count','sidebar_color','tweet_coord','tweet_count','tweet_created','tweet_id','tweet_location','user_timezone'], axis = 1) 

In [5]:
df = df[df['gender:confidence'] == 1]

In [6]:
male = df[df['gender'] == 'male']
female = df[df['gender'] == 'female']
df = male.append(female)

In [7]:
df.head()

,gender,gender:confidence,description,text
0,male,1.0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,male,1.0,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
3,male,1.0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
7,male,1.0,The secret of getting ahead is getting started.,Gala Bingo clubs bought for å£241m: The UK's l...
17,male,1.0,"Over enthusiastic F1 fan. Model collector, mu...",@coolyazzy94 Ditto - I'm still learning the fa...


In [8]:
df['description'] = df['description'].astype(str)
df['text'] = df['text'].astype(str)

In [9]:
df['data'] = df[['description', 'text']].apply(lambda x: ' '.join(x), axis=1)

In [10]:
df = df.reset_index()

In [11]:
del df['index']

In [12]:
df.head()

,gender,gender:confidence,description,text,data
0,male,1.0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm. Robbie E Responds To Cri...
1,male,1.0,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,I'm the author of novels filled with family dr...
2,male,1.0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
3,male,1.0,The secret of getting ahead is getting started.,Gala Bingo clubs bought for å£241m: The UK's l...,The secret of getting ahead is getting started...
4,male,1.0,"Over enthusiastic F1 fan. Model collector, mu...",@coolyazzy94 Ditto - I'm still learning the fa...,"Over enthusiastic F1 fan. Model collector, mu..."


In [13]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [14]:
lemma = WordNetLemmatizer()

In [15]:
corpus = []
for i in range(0,len(df)):
    words = re.sub('[^a-zA-Z]',' ',df['data'][i])
    words = words.lower()
    words = words.split()
    words = [lemma.lemmatize(word) for word in words if word not in stopwords.words('english')]
    words = ' '.join(words)
    corpus.append(words)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfvec= TfidfVectorizer(max_features=5000)

In [17]:
x = tfvec.fit_transform(corpus).toarray()

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [19]:
df['gender_label'] = le.fit_transform(df['gender'])

In [20]:
y = df[['gender_label']].values

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8)

In [35]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8016, 5000), (2004, 5000), (8016, 1), (2004, 1))

In [36]:
from sklearn.naive_bayes import MultinomialNB

In [37]:
gender_detect_nb = MultinomialNB().fit(x_train,y_train)

c:\users\shashwat agarwal\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [38]:
y_pred_nb = gender_detect_nb.predict(x_test)

In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
acurracy_cm = confusion_matrix(y_test,y_pred_nb)

In [41]:
acurracy_cm

array([[861, 171],
       [478, 494]], dtype=int64)

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
accuracy_score(y_test,y_pred_nb)

0.6761477045908184